In [52]:
import pip

def install(package):
    pip.main(['install', package])

# Example
if __name__ == '__main__':
    install('pydicom')

You are using pip version 9.0.1, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import dicom
import os, glob
import scipy.ndimage
import matplotlib.pyplot as plt

from skimage import measure, morphology
from PIL import Image
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
from scipy.ndimage.morphology import binary_dilation,generate_binary_structure
from skimage.morphology import convex_hull_image
from scipy.ndimage.interpolation import zoom
from scipy.io import loadmat
import warnings
from multiprocessing import Pool, cpu_count
from functools import partial


%matplotlib inline


#/usr/local/kdeploy/Tensorflow/DATA/NLST_trial/217676
#/usr/local/kdeploy/Tensorflow/DATA/NLST_trial/218662

#Path to specific patient's folder
Patient_PATH = "/usr/local/kdeploy/Tensorflow/DATA/NLST_trial/218662"
#Series ID
Patient_SERIES = "1.3.6.1.4.1.14519.5.2.1.7009.9004.125728469265057104905925332880"


#217676.s3.19990102-GE MEDICAL SYSTEMS-1.25-120-STANDARD-80.1.3.6.1.4.1.14519.5.2.1.7009.9004.238276364730046742982927193825.v10.new.png'
#Path to mask image for that patient
mask_image = '218662.s3.19990102-SIEMENS-1-120-B30f-250.1.3.6.1.4.1.14519.5.2.1.7009.9004.125728469265057104905925332880.v12.new.png'


txt_files = glob.glob(os.path.join(Patient_PATH,"*.txt"))
series_image_dict = None
with open(txt_files[0]) as dicom_md:
   for line in dicom_md:
       if series_image_dict is None:
           series_image_dict = {}
           continue
       line_split = line.split(" ")
       series, image =  line_split[2], line_split[3]
       if series not in series_image_dict.keys():
          series_image_dict[series] = []
       series_image_dict[series].append(os.path.basename(image).strip())
    
filter_series = series_image_dict[Patient_SERIES]

In [ ]:
patients = os.listdir(Patient_PATH)
patient_0_path = os.path.join(PATH,patients[0])


def load_scan(path):
    slices = [dicom.read_file(path + '/' + s) for s in os.listdir(path) if s in filter_series]
    slices.sort(key = lambda x: float(x.ImagePositionPatient[2]))
    if slices[0].ImagePositionPatient[2] == slices[1].ImagePositionPatient[2]:
        sec_num = 2;
        while slices[0].ImagePositionPatient[2] == slices[sec_num].ImagePositionPatient[2]:
            sec_num = sec_num+1;
        slice_num = int(len(slices) / sec_num)
        slices.sort(key = lambda x:float(x.InstanceNumber))
        slices = slices[0:slice_num]
        slices.sort(key = lambda x:float(x.ImagePositionPatient[2]))
    try:
        slice_thickness = np.abs(slices[0].ImagePositionPatient[2] - slices[1].ImagePositionPatient[2])
    except:
        slice_thickness = np.abs(slices[0].SliceLocation - slices[1].SliceLocation)
        
    for s in slices:
        s.SliceThickness = slice_thickness
        
    return slices

def get_pixels_hu(slices):
    image = np.stack([s.pixel_array for s in slices])
    # Convert to int16 (from sometimes int16), 
    # should be possible as values should always be low enough (<32k)
    image = image.astype(np.int16)
    
    # Convert to Hounsfield units (HU)
    for slice_number in range(len(slices)):        
        intercept = slices[slice_number].RescaleIntercept
        slope = slices[slice_number].RescaleSlope
        
        if slope != 1:
            image[slice_number] = slope * image[slice_number].astype(np.float64)
            image[slice_number] = image[slice_number].astype(np.int16)
            
        image[slice_number] += np.int16(intercept)
    
    return np.array(image, dtype=np.int16), np.array([slices[0].SliceThickness] + slices[0].PixelSpacing, dtype=np.float32)



In [ ]:
patient_0_image, patient_0_spacing = get_pixels_hu(load_scan(patient_0_path))
spacing = patient_0_spacing
def plot_3d(image, overlay_img, threshold=-300):
    
    # Position the scan upright, 
    # so the head of the patient would be at the top facing the camera
    p = image.transpose(2,1,0)
    
    #Reduce computational power
    #p = measure.block_reduce(p,(3,3,3), func=np.max)
    
    verts, faces, _, _ = measure.marching_cubes(p, threshold)

    fig = plt.figure(figsize=(10, 10))
    ax = fig.add_subplot(111, projection='3d')

    # Fancy indexing: `verts[faces]` to generate a collection of triangles
    mesh = Poly3DCollection(verts[faces], alpha=0.70)
    face_color = [0.45, 0.45, 0.75]
    mesh.set_facecolor(face_color)
    ax.add_collection3d(mesh)

    ax.set_xlim(0, p.shape[0])
    ax.set_ylim(0, p.shape[1])
    ax.set_zlim(0, p.shape[2])
    
    #Add overlay
    overlay = overlay_img.transpose(2,1,0)
    nodules3 = np.where(overlay==255)
    ax.scatter(nodules3[0], nodules3[1], nodules3[2], zdir='z', c= 'red')
    
    plt.show()
    

In [ ]:
%matplotlib inline
import numpy as np
from scipy import misc
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

im = misc.imread(mask_image)
im3 = np.reshape(im,(int(im.shape[0]/512),512,im.shape[1]))
nodules3 = np.where(im3==255)

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(nodules3[0], nodules3[1], nodules3[2], zdir='z', c= 'red')


In [ ]:
plot_3d(patient_0_image, im3, 400)